<a href="https://colab.research.google.com/github/austek/My-Colab-Notebooks/blob/main/Piper/Piper_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0) Colab runtime + GPU check

In [1]:
# Check GPU
import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

Python: 3.12.12
PyTorch: 2.9.0+cu126
CUDA available: True
Fri Nov 28 12:17:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |              

1) System packages (incl. eSpeak dev)

In [2]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
!pkg-config --modversion espeak-ng

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,491 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,008 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu ja

2) Clone repo fresh

In [3]:
%cd /content
!rm -rf piper1-gpl
!git clone https://github.com/OHF-voice/piper1-gpl.git
%cd piper1-gpl
!pwd

/content
Cloning into 'piper1-gpl'...
remote: Enumerating objects: 817, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 817 (delta 234), reused 207 (delta 207), pack-reused 527 (from 2)
Receiving objects: 100% (817/817), 4.69 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (460/460), done.
/content/piper1-gpl
/content/piper1-gpl


3) Python deps (editable install, no venv in Colab)

In [4]:
!python3 -m pip install --upgrade pip setuptools wheel
!python3 -m pip install -e ".[train]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.0 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
Obtaining file:///content/piper1-gpl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/84

4) Build the Cython extension used for alignment

In [5]:
%cd /content/piper1-gpl
!chmod +x ./build_monotonic_align.sh
!./build_monotonic_align.sh

/content/piper1-gpl
Compiling /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
P

5) Dev build (repo mode)

In [6]:
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja

  Using cached scikit_build-0.18.1-py3-none-any.whl.metadata (18 kB)
  Using cached cmake-4.2.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.5 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached scikit_build-0.18.1-py3-none-any.whl (85 kB)
Using cached cmake-4.2.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (28.9 MB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Attempting uninstall: cmake
    Found existing installation: cmake 3.31.10
    Uninstalling cmake-3.31.10:
      Successfully uninstalled cmake-3.31.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [cmake]


In [1]:
%cd /content/piper1-gpl
!pip install -v .
!python3 setup.py build_ext --inplace -v

[Errno 2] No such file or directory: '/content/piper1-gpl'
/content
Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


6) (Optional) Mount Google Drive for datasets and outputs

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


7) Set paths and training hyperparams

In [9]:
from pathlib import Path

# ==== CHANGE THESE ====
VOICE_NAME      = "my_colab_voice"
ESPEAK_VOICE    = "en-us"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 22050
BATCH_SIZE      = 32       # drop to 8 or 4 if you OOM

DATA_ROOT       = Path("/content/drive/MyDrive/tts_data/myvoice")
AUDIO_DIR       = DATA_ROOT / "wavs"
CSV_PATH        = DATA_ROOT / "metadata.csv"

CACHE_DIR       = Path("/content/piper_cache")
CONFIG_PATH     = DATA_ROOT / f"{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = ""  # e.g., "/content/drive/MyDrive/piper_ckpts/en_US-lessac-medium.ckpt"

# Make sure dirs exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print("CSV exists:", CSV_PATH.exists())
print("Audio dir exists:", AUDIO_DIR.exists())
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

CSV exists: True
Audio dir exists: True
Cache dir: /content/piper_cache
Config will be written to: /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json


8) Quick sanity checks

In [10]:
!espeak-ng --voices | head -n 20

Pty Language       Age/Gender VoiceName          File                 Other Languages
 5  af              --/M      Afrikaans          gmw/af               
 5  am              --/M      Amharic            sem/am               
 5  an              --/M      Aragonese          roa/an               
 5  ar              --/M      Arabic             sem/ar               
 5  as              --/M      Assamese           inc/as               
 5  az              --/M      Azerbaijani        trk/az               
 5  ba              --/M      Bashkir            trk/ba               
 5  bg              --/M      Bulgarian          zls/bg               
 5  bn              --/M      Bengali            inc/bn               
 5  bpy             --/M      Bishnupriya_Manipuri inc/bpy              
 5  bs              --/M      Bosnian            zls/bs               
 5  ca              --/M      Catalan            roa/ca               
 5  cmn             --/M      Chinese_(Mandarin) sit/cmn    

In [12]:
import pandas as pd, io, os, textwrap

csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df.head())
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

   audio                                               text
0  1.wav  Çavê xwe ji rengê şînê Ezmanî girtiye... Weke ...
Missing among first 5: []


9) Kick off training

In [13]:
!python3 -m piper.train fit \
  --data.voice_name "my_colab_voice" \
  --data.csv_path "/content/drive/MyDrive/tts_data/myvoice/metadata.csv" \
  --data.audio_dir "/content/drive/MyDrive/tts_data/myvoice/wavs" \
  --model.sample_rate 22050 \
  --data.espeak_voice "en-us" \
  --data.cache_dir "/content/piper_cache" \
  --data.config_path "/content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json" \
  --data.batch_size 8 \
  --ckpt_path "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)
/usr/local/lib/python3.12/dist-packages/lightning/fabric/utilities/seed.py:44: No seed found, seed set to 0
Seed set to 0
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
💡 Tip: For seamless cl

10. Export to onnx

In [ ]:
!python3 -m piper.train.export_onnx \
  --checkpoint "/content/piper1-gpl/lightning_logs/version_2/checkpoints/epoch=2174-step=2680.ckpt" \
  --output-file "/content/model.onnx"

In [ ]:
!cp /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json /content/model.onnx.json